In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.utils import np_utils
from keras.metrics import categorical_accuracy
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
labels = pd.read_csv('labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
y = labels['breed']
print(y[0:5])
classes = len(labels['breed'].unique())
print(classes)
y_lookup = np.eye(classes)
print(y_lookup[0,])

0         boston_bull
1               dingo
2            pekinese
3            bluetick
4    golden_retriever
Name: breed, dtype: object
120
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [4]:
labels_int = LabelEncoder()
y_ints = labels_int.fit_transform(y)
y_ints = y_ints.reshape(len(y_ints),1)
print(y_ints.shape)
y_onehot = [y_lookup[i] for i in y_ints]
# labels_onehot = OneHotEncoder(sparse=False)
# y_onehot = labels_onehot.fit_transform(y_ints)
print(np.array(y_onehot).shape)

(10222, 1)
(10222, 1, 120)


In [5]:
y = np.array(y)
item_int_lkup = pd.DataFrame({'int': y_ints.ravel(), 'label': y.ravel()})
item_int_lkup = item_int_lkup.drop_duplicates()
# print(item_int_lkup)
item_int_lkup = dict(zip(item_int_lkup.int, item_int_lkup.label))
item_int_lkup
print(item_int_lkup.get(0))

affenpinscher


In [6]:
y_onehot = np.reshape(y_onehot, (len(y_onehot), classes))

In [8]:
train_path = './train/'
test_path = './test/'
X = []
for i,row in labels.iterrows():
    image = Image.open(train_path+row['id']+'.jpg')
    image = image.resize((256, 256), Image.ANTIALIAS)
    # image = image.convert('LA')
    image = np.array(image)
    image = image/255
    # image = image[:,:,0]
    # image = image.reshape((image.shape[0],image.shape[1],1))
    X.append(image)

In [9]:
X = np.array(X, np.float32)
print(X.shape)

(10222, 256, 256, 3)


In [45]:
def create_model(layers_to_freeze = 8, cnn = 'xception', classes=5):
    inputs = Input(shape=X.shape[1:4])
    bn = BatchNormalization()(inputs)
    if cnn=='xception':
        model = Xception(weights='imagenet', include_top=False)
    if cnn=='inception':
        model = InceptionV3(weights='imagenet', include_top=False)
    if cnn=='vgg19':
        model = VGG19(weights='imagenet', include_top=False)
    if cnn=='vgg16':
        model = VGG16(weights='imagenet', include_top=False)
    if cnn=='resnet':
        model = ResNet50(weights='imagenet', include_top=False)
    if cnn=='mobilenet':
        model = MobileNet(weights='imagenet', include_top=False)
    for layer in model.layers[:layers_to_freeze]:
        layer.trainable = False
    for layer in model.layers[layers_to_freeze:]:
        layer.trainable = True
    x = model(bn)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.1)(x)
    x = Dense(classes, activation='softmax')(x)
    model_final = Model(input=inputs, output=x)
    model_final.compile(loss='categorical_crossentropy', metrics=["accuracy", categorical_accuracy], optimizer='sgd')
    return model_final

model = create_model(10, 'xception', classes)

C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [14]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

x_train, x_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

data_gen = datagen_train = ImageDataGenerator(
    #rescale=1.0/255, 
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip=True
                                  )

data_gen.fit(x_train)

generator = data_gen.flow(x_train, y_train, batch_size=32)

In [46]:
data_gen = datagen_train = ImageDataGenerator(
    #rescale=1.0/255, 
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip=True
                                  )

data_gen.fit(X)

generator = data_gen.flow(X, y_onehot, batch_size=32)

In [47]:
model_path = './best_model.hdf5'

In [48]:
callbacks = [# EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto'),
    TQDMNotebookCallback(leave_inner=True, leave_outer=True),
            ModelCheckpoint(model_path, monitor='val_categorical_accuracy', verbose=0, save_best_only=True, mode='max'),
]
train_performance = model.fit_generator(generator, steps_per_epoch=16, epochs= 250, verbose=1, 
                                      callbacks=callbacks,
                                      validation_data=(x_test, y_test)
                                     )

Epoch 1/250


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Anaconda\envs\py35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Anaconda\envs\py35\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Anaconda\envs\py35\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



16/16 [==============================] - ETA: 196s - loss: 4.8363 - acc: 0.0000e+00 - categorical_accuracy: 0.0000e+00 - ETA: 105s - loss: 4.8276 - acc: 0.0000e+00 - categorical_accuracy: 0.0000e+00 - ETA: 73s - loss: 4.8491 - acc: 0.0000e+00 - categorical_accuracy: 0.0000e+00  - ETA: 56s - loss: 4.8371 - acc: 0.0000e+00 - categorical_accuracy: 0.0000e+00 - ETA: 45s - loss: 4.8257 - acc: 0.0000e+00 - categorical_accuracy: 0.0000e+00

In [17]:
model.save('ensemble_xception.h5')

In [19]:
import os
X_sub = []
for file in os.listdir(test_path):
    image = Image.open(test_path+file)
    image = image.resize((256, 256), Image.ANTIALIAS)
    image = np.array(image)
    image = image/255
    X_sub.append(image)
X_sub = np.array(X_sub, np.float32)
print(np.shape(X_sub))

(10357, 256, 256, 3)


In [49]:
predictions = model.predict(X_sub, verbose=1)
print(predictions.shape)

10357/10357 [==============================] - ETA: 755s - ETA: 491s - ETA: 408s - ETA: 366s - ETA: 341s - ETA: 321s - ETA: 307s - ETA: 296s - ETA: 288s - ETA: 281s - ETA: 276s - ETA: 271s - ETA: 267s - ETA: 264s - E

In [50]:
reference = pd.DataFrame({'y':y, 'y_ints':y_ints.reshape(y_ints.shape[0])})

In [51]:
sub_files = []
for file in os.listdir(test_path):
    sub_files.append(file.replace('.jpg',''))
submission = pd.DataFrame({'id':sub_files})

In [52]:
reference.drop_duplicates().sort('y_ints')
reference.to_csv('./reference.csv', sep=',', encoding='utf-8', index=False)

C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [53]:
submission['affenpinscher']=predictions[:,0]
submission['afghan_hound']=predictions[:,1]
submission['african_hunting_dog']=predictions[:,2]
submission['airedale']=predictions[:,3]
submission['american_staffordshire_terrier']=predictions[:,4]
submission['appenzeller']=predictions[:,5]
submission['australian_terrier']=predictions[:,6]
submission['basenji']=predictions[:,7]
submission['basset']=predictions[:,8]
submission['beagle']=predictions[:,9]
submission['bedlington_terrier']=predictions[:,10]
submission['bernese_mountain_dog']=predictions[:,11]
submission['black-and-tan_coonhound']=predictions[:,12]
submission['blenheim_spaniel']=predictions[:,13]
submission['bloodhound']=predictions[:,14]
submission['bluetick']=predictions[:,15]
submission['border_collie']=predictions[:,16]
submission['border_terrier']=predictions[:,17]
submission['borzoi']=predictions[:,18]
submission['boston_bull']=predictions[:,19]
submission['bouvier_des_flandres']=predictions[:,20]
submission['boxer']=predictions[:,21]
submission['brabancon_griffon']=predictions[:,22]
submission['briard']=predictions[:,23]
submission['brittany_spaniel']=predictions[:,24]
submission['bull_mastiff']=predictions[:,25]
submission['cairn']=predictions[:,26]
submission['cardigan']=predictions[:,27]
submission['chesapeake_bay_retriever']=predictions[:,28]
submission['chihuahua']=predictions[:,29]
submission['chow']=predictions[:,30]
submission['clumber']=predictions[:,31]
submission['cocker_spaniel']=predictions[:,32]
submission['collie']=predictions[:,33]
submission['curly-coated_retriever']=predictions[:,34]
submission['dandie_dinmont']=predictions[:,35]
submission['dhole']=predictions[:,36]
submission['dingo']=predictions[:,37]
submission['doberman']=predictions[:,38]
submission['english_foxhound']=predictions[:,39]
submission['english_setter']=predictions[:,40]
submission['english_springer']=predictions[:,41]
submission['entlebucher']=predictions[:,42]
submission['eskimo_dog']=predictions[:,43]
submission['flat-coated_retriever']=predictions[:,44]
submission['french_bulldog']=predictions[:,45]
submission['german_shepherd']=predictions[:,46]
submission['german_short-haired_pointer']=predictions[:,47]
submission['giant_schnauzer']=predictions[:,48]
submission['golden_retriever']=predictions[:,49]
submission['gordon_setter']=predictions[:,50]
submission['great_dane']=predictions[:,51]
submission['great_pyrenees']=predictions[:,52]
submission['greater_swiss_mountain_dog']=predictions[:,53]
submission['groenendael']=predictions[:,54]
submission['ibizan_hound']=predictions[:,55]
submission['irish_setter']=predictions[:,56]
submission['irish_terrier']=predictions[:,57]
submission['irish_water_spaniel']=predictions[:,58]
submission['irish_wolfhound']=predictions[:,59]
submission['italian_greyhound']=predictions[:,60]
submission['japanese_spaniel']=predictions[:,61]
submission['keeshond']=predictions[:,62]
submission['kelpie']=predictions[:,63]
submission['kerry_blue_terrier']=predictions[:,64]
submission['komondor']=predictions[:,65]
submission['kuvasz']=predictions[:,66]
submission['labrador_retriever']=predictions[:,67]
submission['lakeland_terrier']=predictions[:,68]
submission['leonberg']=predictions[:,69]
submission['lhasa']=predictions[:,70]
submission['malamute']=predictions[:,71]
submission['malinois']=predictions[:,72]
submission['maltese_dog']=predictions[:,73]
submission['mexican_hairless']=predictions[:,74]
submission['miniature_pinscher']=predictions[:,75]
submission['miniature_poodle']=predictions[:,76]
submission['miniature_schnauzer']=predictions[:,77]
submission['newfoundland']=predictions[:,78]
submission['norfolk_terrier']=predictions[:,79]
submission['norwegian_elkhound']=predictions[:,80]
submission['norwich_terrier']=predictions[:,81]
submission['old_english_sheepdog']=predictions[:,82]
submission['otterhound']=predictions[:,83]
submission['papillon']=predictions[:,84]
submission['pekinese']=predictions[:,85]
submission['pembroke']=predictions[:,86]
submission['pomeranian']=predictions[:,87]
submission['pug']=predictions[:,88]
submission['redbone']=predictions[:,89]
submission['rhodesian_ridgeback']=predictions[:,90]
submission['rottweiler']=predictions[:,91]
submission['saint_bernard']=predictions[:,92]
submission['saluki']=predictions[:,93]
submission['samoyed']=predictions[:,94]
submission['schipperke']=predictions[:,95]
submission['scotch_terrier']=predictions[:,96]
submission['scottish_deerhound']=predictions[:,97]
submission['sealyham_terrier']=predictions[:,98]
submission['shetland_sheepdog']=predictions[:,99]
submission['shih-tzu']=predictions[:,100]
submission['siberian_husky']=predictions[:,101]
submission['silky_terrier']=predictions[:,102]
submission['soft-coated_wheaten_terrier']=predictions[:,103]
submission['staffordshire_bullterrier']=predictions[:,104]
submission['standard_poodle']=predictions[:,105]
submission['standard_schnauzer']=predictions[:,106]
submission['sussex_spaniel']=predictions[:,107]
submission['tibetan_mastiff']=predictions[:,108]
submission['tibetan_terrier']=predictions[:,109]
submission['toy_poodle']=predictions[:,110]
submission['toy_terrier']=predictions[:,111]
submission['vizsla']=predictions[:,112]
submission['walker_hound']=predictions[:,113]
submission['weimaraner']=predictions[:,114]
submission['welsh_springer_spaniel']=predictions[:,115]
submission['west_highland_white_terrier']=predictions[:,116]
submission['whippet']=predictions[:,117]
submission['wire-haired_fox_terrier']=predictions[:,118]
submission['yorkshire_terrier']=predictions[:,119]

In [54]:
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.000016,0.000011,0.000004,0.000006,0.000002,0.000005,0.000007,0.000005,0.000004,...,0.000017,0.000005,0.000005,0.000008,0.000005,0.000005,0.000004,0.000007,0.000003,0.000013
1,00102ee9d8eb90812350685311fe5890,0.000140,0.000131,0.000130,0.000208,0.000074,0.000176,0.000078,0.000094,0.000059,...,0.000360,0.000106,0.000069,0.000070,0.000038,0.000051,0.000322,0.000087,0.000125,0.000175
2,0012a730dfa437f5f3613fb75efcd4ce,0.000017,0.001536,0.000054,0.000020,0.000032,0.000051,0.000024,0.000009,0.000066,...,0.000019,0.000012,0.000020,0.000050,0.000137,0.000325,0.000021,0.000055,0.000026,0.000031
3,001510bc8570bbeee98c8d80c8a95ec1,0.002786,0.000116,0.000138,0.000075,0.000452,0.000104,0.000020,0.000185,0.000102,...,0.000688,0.000265,0.000061,0.000109,0.000096,0.000042,0.000063,0.000206,0.000159,0.000100
4,001a5f3114548acdefa3d4da05474c2e,0.064542,0.000503,0.000206,0.000061,0.000121,0.000230,0.000519,0.000431,0.000268,...,0.000509,0.000227,0.000243,0.000244,0.000213,0.000065,0.000436,0.000417,0.000299,0.004407


In [55]:
submission.to_csv('./submission.csv', sep=',', encoding='utf-8', index=False)